In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# CHSH-Ungleichung

*Nutzungsschätzung: Zwee Minudn aufm Heron r2-Prozessor (HINNWEIS: Das is bloß ne Schätzung. Ihre Laufzeit gann variern.)*
## Hindergrundn
In dem Tutorial machn mer'n Experiment aufm Quanterechner, um de Verletzung von de CHSH-Ungleichung mitn Estimator-Primitiv zu zeechn.

De CHSH-Ungleichung, benannt nach de Autorn Clauser, Horne, Shimony un Holt, werd genutzt, um Bells Theorem (1969) experimentell zu beweis'n. Das Theorem sacht aus, dass lokale Theorieen mit verborgne Variabln nich alle Gonsequenzn von de Verschränkung in de Quantemechanik erglärn gönn'. De Verletzung von de CHSH-Ungleichung werd genutzt, um zu zeechn, dass de Quantemechanik mit lokale Theorieen mit verborgne Variabln nich vereinbar is. Das is'n wichtichs Experiment för's Verständnis von de Grundlagn von de Quantemechanik.

De Nobelbreis för Physig 2022 wor an Alain Aspect, John Clauser un Anton Zeilinger vergeem worden, unnerm annern för ihre Pionierarbeit in de Quanteinformationswissenschaft un besonders för ihre Experimende mit verschrängte Photonen, de de Verletzung von de Bellschn Ungleichungn demonstriert ham.
## Anforderungn
Bevorn Se mit dem Tutorial anfangn, stelln Se sicher, dass Se Folgends installiert ham:

* Qiskit SDK v1.0 oder neier, mit [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization)-Unnerstützung
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 oder neier
## Eirichtung

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Schritt 1: Glassische Eingabn auf'n Quantebroblem abbild'n
För das Experiment machn mer'n verschränktes Baar, wo mer jedes Qubit auf zwee verschiedne Basn mess'n. Mer bezeechn' de Basn förn erschte Qubit mit $A$ un $a$ un de Basn förn zweete Qubit mit $B$ un $b$. Das erlaabt's uns, de CHSH-Größe $S_1$ zu berechnen:

$$
S_1 = A(B-b) + a(B+b).
$$

Jede Observable is entweder $+1$ oder $-1$. Glar is, dass eener von de Terme $B\pm b$ gleich $0$ sein muss un de annere $\pm 2$ sein muss. Deshalb is $S_1 = \pm 2$. De Durchschnittswert von $S_1$ muss de Ungleichung erfülln:

$$
|\langle S_1 \rangle|\leq 2.
$$

Wennmer $S_1$ in Bezug auf $A$, $a$, $B$ un $b$ auswiggeln, griechn mer:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Se gönn' noch ne weitere CHSH-Größe $S_2$ definieren:

$$
S_2 = A(B+b) - a(B-b),
$$

Das führt zu ner weitern Ungleichung:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Wenn de Quantemechanik durch lokale Theorieen mit verborgne Variabln beschriebn wern gann, müssn de vorischn Ungleichungn wohr sein. Aber wie in dem Tutorial gezeecht werd, gönn' diese Ungleichungn aufm Quanterechner verletzt wern. Deshalb is de Quantemechanik nich mit lokale Theorieen mit verborgne Variabln vereinbar.
Falls Se mehr Theorie lern wolln, gugn Se sich [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) mitn John Watrous an.
Mer machn'n verschränktes Baar zwischn zwee Qubits in nem Quanterechner, indem mer'n Bell-Zustand $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ erzeugn. Mitn Estimator-Primitiv gönn' Se direggt de nötichn Erwartungswerte ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$ un $\langle ab \rangle$) griechn, um de Erwartungswerte von de beedn CHSH-Größn $\langle S_1\rangle$ un $\langle S_2\rangle$ zu berechnen. Vorn Einführung vom Estimator-Primitiv häddn Se de Erwartungswerte aus de Messergeebnisse konstruiern müss'n.

Mer mess'n das zweete Qubit in de $Z$- un $X$-Basn. Das erschte Qubit werd aaoch in orthogonale Basn gemess'n, aber mitn Winkel bezüchlich vom zweeten Qubit, den mer zwischn $0$ un $2\pi$ variern wern. Wie Se sehn wern, macht das Estimator-Primitiv das Ausführn von parametrisierten Schaltgreesn sehr eenfach. Anstatt ne Reihe von CHSH-Schaltgreesn zu machn, müssn Se bloß *eenen* CHSH-Schaltgrees mitn Parameter machn, der'n Messwinkel angibt, un ne Reihe von Phasenwerte förn Parameter.

Schließlich wern mer de Ergeebnisse analysiern un gegn Messwinkel auftraan. Se wern sehn, dass för'n bestimmdn Bereich von Messwinkeln de Erwartungswerte von de CHSH-Größn $|\langle S_1\rangle| > 2$ oder $|\langle S_2\rangle| > 2$ sin, was de Verletzung von de CHSH-Ungleichung demonstriert.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### 'N parametrisierten CHSH-Schaltgrees machn
Zuerst schreebmer'n Schaltgrees mitn Parameter $\theta$, den mer `theta` nenn'. Das [`Estimator`-Primitiv](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) gann'n Schaltgreesaufbau un de Ausgabeanalyse riesig vereinfachen, indemses direggt Erwartungswerte von Observabln liefert. Viele interessande Probleme, besonders för kurzfristige Anwendungn auf verrauschte Systeme, gönn' in Form von Erwartungswerte formuliert wern. Das `Estimator` (V2)-Primitiv gann automatisch de Messbasis basiernd auf de bereetgestellte Observable ännern.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### 'Ne Lischde von Phasenwerte machn, de schbäder zugewiesen wern
Nachdem Se'n parametrisierten CHSH-Schaltgrees gemacht ham, machn Se ne Lischde von Phasenwerte, de dem Schaltgrees im nächsten Schritt zugewiesen wern. Se gönn' folgenden Code nutz'n, um ne Lischde von 21 Phasenwerte im Bereich von $0$ bis $2 \pi$ mit gleichm Abstand zu machn, also $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observabln
Jetzt brauchn mer Observabln, aus denen mer de Erwartungswerte berechnen gönn'. In unserm Fall betrachtn mer orthogonale Basn för jedes Qubit, wobei de parametrisierte $Y$-Rotation förn erschte Qubit de Messbasis beinah gontinuierlich bezüchlich von de Basis vom zweeten Qubit variiert. Mer wähln deshalb de Observabln $ZZ$, $ZX$, $XZ$ un $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Schritt 2: Broblem för de Ausführung auf Quantehardware obtimiern

Um de Gesamtausführungszeet vom Job zu reduzieren, aggzebtiern V2-Primitive bloß Schaltgreese un Observabln, de de vom Zielsystem unnerstützten Anweisungn un de Gonneggtivität entsprechn (bezeechnet als Instruction Set Architecture (ISA)-Schaltgreese un -Observabln).

### ISA-Schaltgrees

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA-Observabln

Ebenfalls müssn mer de Observabln transformiern, um se backend-gompatibel zu machn, bevorn mer Jobs mit [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run) ausführn. Mer gönn' de Transformation mit de `apply_layout`-Methode vom `SparsePauliOp`-Objeggd durchführn.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Schritt 3: Ausführn mit Qiskit-Primitive
Um das gesambte Experiment in eenm einzichn Aufruf vom [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) auszuführn.
Mer gönn'n [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2)-Primitiv machn, um unsere Erwartungswerte zu berechnen. De `EstimatorV2.run()`-Methode nimmt'n Iterable von `primitive unified blocs (PUBs)`. Jedes PUB is'n Iterable im Format `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)